# Faims Analysis

This notebook covers the following things:
- processing
- xirt
- qc

In [2]:
import sys
sys.path.append("../") # go to parent dir

from xirt import __main__ as xm
from xirt import processing as xirtp
from xirt import predictor as xr
from xirt import xirtnet
from xifaims import processing as xp
import yaml
import numpy as np
import pandas as pd

infile = "../data/4PM_DSS_LS_nonunique1pCSM.csv"
xirt_loc = "../parameters/xirt_faims.yaml"
setup_loc = "../parameters/xirt_learning.yaml"
xirt_params = yaml.load(open(xirt_loc), Loader=yaml.FullLoader)
learning_params = yaml.load(open(setup_loc), Loader=yaml.FullLoader)
matches_df = pd.read_csv(infile)
matches_df["cv"] = matches_df["run"].apply(xp.get_faims_cv)
matches_df["score"] = matches_df["Score"]
matches_df = matches_df.rename(columns={"PepSeq1": "Peptide1", "PepSeq2": "Peptide2", "fdr": "FDR"})
print(np.unique(matches_df["cv"]))
print(len(np.unique(matches_df["cv"])))
matches_df

[ 20.  25.  30.  35.  40.  45.  50.  55.  60.  65.  70.  75.  80.  85.
  90.  95. 100. 110.]
18


,SearchID,PSMID,run,exp charge,exp m/z,exp mass,exp fractionalmass,match charge,match mass,match fractionalmass,...,Protein1FDR,Protein2FDR,LinkFDR,PPIFDR,peptide pair id,link id,ppi id,info,cv,score
0,16099,21096680379,B200211_14_Lumos_LS_IN_130_4PM_DSS_FAIMS_CV055...,5,726.217123,3626.049233,0.049233,5,3625.046777,0.046777,...,0.0,0.00,0.000,0.00,4692.0,499.0,28.0,NaN,55.0,21.997
1,16099,21096804843,B200211_18_Lumos_LS_IN_130_4PM_DSS_FAIMS_CV040...,4,907.269344,3625.048270,0.048270,4,3625.046777,0.046777,...,0.0,0.00,0.000,0.00,4692.0,499.0,28.0,NaN,40.0,21.909
2,16100,21101030038,B191129_15_Lumos_LS_IN_130_4PM_DSS_FAIMS_CV040...,4,1247.645559,4986.553130,0.553130,4,4986.549933,0.549933,...,0.0,0.00,0.000,0.00,6508.0,1484.0,13.0,NaN,40.0,21.510
3,16100,21101073648,B191129_18_Lumos_LS_IN_130_4PM_DSS_FAIMS_CV045...,4,1247.896860,4987.558336,0.558336,4,4986.549933,0.549933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,21.256
4,16099,21096722391,B200211_15_Lumos_LS_IN_130_4PM_DSS_FAIMS_CV050...,5,726.417035,3627.048791,0.048791,5,3625.046777,0.046777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,21.157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9174,16099,21096749308,B200211_15_Lumos_LS_IN_130_4PM_DSS_FAIMS_CV050...,6,619.012121,3708.029068,0.029068,6,3705.014322,0.014322,...,0.0,0.00,0.000,0.00,6.0,94.0,2.0,NaN,50.0,1.751
9175,16100,21101066232,B191129_18_Lumos_LS_IN_130_4PM_DSS_FAIMS_CV045...,4,904.947846,3615.762279,0.762279,4,3615.763403,0.763403,...,1.0,0.00,0.060,0.75,5.0,4.0,1.0,NaN,45.0,1.745
9176,16099,21096618286,B200211_10_Lumos_LS_IN_130_4PM_DSS_FAIMS_CV070...,4,803.934414,3211.708552,0.708552,4,3211.712219,0.712219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,1.744
9177,16099,21096687067,B200211_14_Lumos_LS_IN_130_4PM_DSS_FAIMS_CV055...,5,799.556549,3992.746360,0.746360,5,3991.747161,0.747161,...,0.5,0.00,0.060,0.50,3.0,2.0,0.0,NaN,55.0,1.736


In [36]:
training_data = xr.preprocess(matches_df, sequence_type="crosslink", max_length=-1, cl_residue=True, fraction_cols=["cv"])
training_data.set_fdr_mask(fdr_cutoff=0.05, str_filter="")

In [138]:
# init the network
xirtnetwork = xirtnet.xiRTNET(xirt_params, input_dim=training_data.features1.shape[1])
training_data.features1
frac_cols = sorted([xirtnetwork.output_p[tt.lower() + "-column"] for tt in xirt_params["predictions"]["fractions"]])
cont_cols = sorted([xirtnetwork.output_p[tt.lower() + "-column"] for tt in xirt_params["predictions"]["continues"]])
print(frac_cols)
print(cont_cols)

[]
['cv']


In [139]:
# do CV
training_data.set_unique_shuffled_sampled_training_idx()
n_splits = 3
test_size = 0.1
for train_idx, val_idx, pred_idx in training_data.iter_splits(n_splits=n_splits, test_size=test_size):
    xirtnetwork.build_model(siamese=xirt_params["siamese"]["use"])
    xirtnetwork.compile()

C:\Users\hanjo\anaconda3\lib\site-packages\xirt\predictor.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cv_locs = np.array([i[1] for i in kf.split(self.features1.loc[self.train_idx])])


In [140]:
cv_counter = 0
callback_path = ""
xirtnetwork.build_model(siamese=xirt_params["siamese"]["use"])
xirtnetwork.compile()
callbacks = xirtnetwork.get_callbacks(suffix=str(cv_counter).zfill(2))
xirtnetwork.model.summary()

Model: "functional_107"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_53 (InputLayer)           [(None, 45)]         0                                            
__________________________________________________________________________________________________
input_54 (InputLayer)           [(None, 45)]         0                                            
__________________________________________________________________________________________________
siamese (Functional)            (None, 100)          33250       input_53[0][0]                   
                                                                 input_54[0][0]                   
__________________________________________________________________________________________________
add_26 (Add)                    (None, 100)          0           siamese[0][0]       

In [141]:
xt_cv = training_data.get_features(training_data.psms.index)
yt_cv = training_data.get_classes(training_data.psms.index, frac_cols=frac_cols, cont_cols=cont_cols)
print(yt_cv)
print(len(xt_cv[0]))
print(len(yt_cv[0]))


[array([55., 40., 40., ..., 70., 55., 45.])]
9179
9179


In [142]:
history = xirtnetwork.model.fit(xt_cv, yt_cv, validation_data=0.1, epochs=xirt_params["learning"]["epochs"],
                                batch_size=xirt_params["learning"]["batch_size"],
                                verbose=1, callbacks=callbacks)

Epoch 1/50
36/36 [==============================] - ETA: 0s - loss: 2173.7673 - mse: 2172.9214

IndexError: list index out of range